# nuImages devkit tutorial

Welcome to the nuImages tutorial.
This demo assumes the database itself is available at `/data/sets/nuimages`.

## A Gentle Introduction to nuImages

In this part of the tutorial, let us go through a top-down introduction of our database. Our dataset is structured as a relational database with tables, tokens and foreign keys. The tables are the following:

1. `image` - An annotated camera image with an associated timestamp.
2. `log` - Log information from which the image was extracted.
3. `camera` - Information about the camera the image was taken from. Intrinsic parameters will be added in future releases.
4. `category` - Taxonomy of object and surface categories (e.g. vehicle.car, flat.drivable_surface). 
5. `attribute` - Property of an object that can change while the category remains the same.
6. `object_ann` - Bounding box and mask annotation of an object (e.g. car, adult).
7. `surface_ann` - Mask annotation of a surface (e.g. driveable surface and ego vehicle).

## nuImages Basics


### Initialization
To initialize the dataset class, we run the following:

In [ ]:
from nuimages import NuImages

nuim = NuImages(verbose=True, lazy=True)

### Tables

As described above, the NuScenes class holds several tables. Each table is a list of records, and each record is a dictionary. For example the first record of the category table is stored at:

In [ ]:
nuim.category[0]

To see the list of all tables, simply refer to the `table_names` variable:

In [ ]:
nuim.table_names

### Lazy loading

Initializing the NuImages instance above was very fast, as we did not actually load the tables. Rather, the class implements lazy loading that overwrites the internal `__getattr__()` function to load a table if it is not already stored in memory. The moment we accessed `category`, we could see the table being loaded from disk. To disable such notifications, just set `verbose=False` when initializing the NuImages object. Furthermore lazy loading can be disabled with `lazy=False`.

### Indexing

Since all tables are lists of dictionaries, we can use standard Python oeperations on them. A very common operation is to retrieve a particular record by its token. Since this operation takes linear time, we precompute an index that helps to access a record in constant time.

Assume we have an image with token `00022ed4280148638a327eb7f23f4836`. To get the corresponding record, we do:

In [ ]:
image = nuim.get('image', '00022ed4280148638a327eb7f23f4836')
image

What this does is actually to lookup the index:

In [ ]:
image_idx = nuim.getind('image', '00022ed4280148638a327eb7f23f4836')

Indeed we can verify that the element with index 3 matches:

In [ ]:
image_ref = nuim.image[image_idx]
assert image == image_ref

### Rendering

To render an image we use the `render_image()` function. We can see the boxes and masks for each object category, as well as the surfaces masks for ego vehicle and driveable surface. At the top left corner of each box, we see the name of the object category. We use the following colors:
- vehicles: magenta
- bikes and bike racks: red
- pedestrians: blue
- cones and barriers: black
- driveable surface and others: orange

In [ ]:
im = nuim.render_image(nuim.image[10]['token'])

### Statistics

The `list_*()` methods are useful to get an overview of the dataset dimensions. Note that these statistics are always *for the current split* that we initialized the `NuImages` instance with, rather than the entire dataset.

In [ ]:
nuim.list_logs()

`list_categories()` lists the category frequencies, as well as the category name and description. Each category is either an object or a surface, but not both.

In [ ]:
nuim.list_categories()

We can also specify an `image_tokens` parameter for `list_categories()` to get the category statistics for a particular image.

In [ ]:
image_tokens = [nuim.image[10]['token']]
nuim.list_categories(image_tokens=image_tokens)

`list_attributes()` shows the frequency, name and description of all attributes:

In [ ]:
nuim.list_attributes()

`list_cameras()` shows us how many camera entries and images there are for each channel, such as the front camera.
Each camera uses slightly different intrinsic parameters, which will be provided in a future release.

In [ ]:
nuim.list_cameras()